In [1]:
from openai import OpenAI
import os

In [2]:
from dotenv import load_dotenv, find_dotenv

def get_api_key():
    _ = load_dotenv(find_dotenv()) # read local .env file
    return os.getenv('OPENAI_API_KEY')

In [3]:
client = OpenAI(
    api_key=get_api_key(),
)

In [4]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=False,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [5]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To achieve desired outputs from a model, provide clear, specific instructions, understanding that longer prompts often offer more clarity and context, thus reducing the likelihood of irrelevant or incorrect responses.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [6]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
  {
    "book_id": "1",
    "title": "The Echoes of Forgotten Dreams",
    "author": "Amanda Sterling",
    "genre": "Fantasy"
  },
  {
    "book_id": "2",
    "title": "The Last Symphony of Silence",
    "author": "Jonathan Kellerman",
    "genre": "Mystery"
  },
  {
    "book_id": "3",
    "title": "Beneath the Shadows of Destiny",
    "author": "Robert Patterson",
    "genre": "Thriller"
  }
]


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [7]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, you can add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


#### Tactic 4: "Few-shot" prompting

In [8]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The mightiest oak in the forest was once a tiny acorn that held its ground; the most majestic mountain withstands the harshest storms yet stands tall; the most beautiful diamond was once a lump of coal that persevered under pressure.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [9]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - In a village, siblings Jack and Jill go on a quest to fetch water from a hilltop well, but they trip and fall, returning home slightly battered but with their adventurous spirits undimmed.
2 - Dans un village, les frères et sœurs Jack et Jill partent en quête pour chercher de l'eau à un puits en haut d'une colline, mais ils trébuchent et tombent, rentrant chez eux légèrement écorchés mais avec leurs esprits aventureux non diminués.
3 - Jack, Jill
4 - {"french_summary": "Dans un village, les frères et sœurs Jack et Jill partent en quête pour chercher de l'eau à un puits en haut d'une colline, mais ils trébuchent et tombent, rentrant chez eux légèrement écorchés mais avec leurs esprits aventureux non diminués.", "num_names": 2}


#### Ask for output in a specified format

In [10]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Siblings Jack and Jill embark on a quest to fetch water from a hilltop well, but trip and fall, returning home slightly battered but with their adventurous spirits undimmed.
Translation: Les frères et sœurs Jack et Jill se lancent dans une quête pour aller chercher de l'eau à un puits en haut d'une colline, mais trébuchent et tombent, rentrant chez eux légèrement écorchés mais avec leur esprit aventureux intact.
Names: Jack, Jill
Output JSON: {"french_summary": "Les frères et sœurs Jack et Jill se lancent dans une quête pour aller chercher de l'eau à un puits en haut d'une colline, mais trébuchent et tombent, rentrant chez eux légèrement écorchés mais avec leur esprit aventureux intact.", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [11]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [12]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
Incorrect


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [13]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a technologically advanced oral care tool designed to provide a superior brushing experience. It features an ultra-slim design that makes it easy to handle and maneuver around the mouth. 

The toothbrush is equipped with smart technology that can track your brushing habits and provide feedback on areas you may be missing or not spending enough time on. This data can be accessed through a mobile app, allowing you to monitor and improve your oral hygiene habits over time.

The AeroGlide UltraSlim Smart Toothbrush also boasts soft, antimicrobial bristles that are gentle on the gums and effective at removing plaque. The bristles are made from a material that resists bacterial growth, ensuring a clean brush every time.

The toothbrush is rechargeable and has a long battery life, making it convenient for daily use and travel. It also has different brushing modes to cater to different oral care needs.

Please note that as of my knowledge upd

# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Generate a marketing product description from a product fact sheet

In [14]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [15]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our mid-century inspired office chair, a stunning addition to our elegant range of office furniture. This chair is not only a stylish choice but also a versatile one, with a variety of shell colors and base finishes to choose from. Whether you prefer the sleek look of stainless steel, the modern appeal of matte black, the clean simplicity of gloss white, or the classic shine of chrome, we've got you covered.

This chair is available with a plastic back and front upholstery (SWC-100) or full upholstery (SWC-110), offering a selection of 10 fabric and 6 leather options. You can also choose to have it with or without armrests, making it a perfect fit for any home or business setting. Plus, it's qualified for contract use.

Built for comfort and durability, this chair features a 5-wheel plastic coated aluminum base and a pneumatic adjust system for easy height adjustment. It measures 53 cm in width, 51 cm in depth, and 80 cm in height, with a seat height of 44 cm and a seat dep

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [16]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Experience the blend of mid-century design and modern comfort with our versatile office chair. Choose from a range of shell colors, base finishes, and upholstery options. Features include adjustable height, optional armrests, and two seat foam densities. Perfect for home or business use. Made in Italy.


## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [17]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

This mid-century inspired office chair features a 5-wheel plastic coated aluminum base and a shell made from cast aluminum with a modified nylon PA6/PA66 coating. It offers a choice of plastic or fully upholstered finishes in a variety of colors, and optional armrests. Suitable for both home and business use.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [18]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
<p>This mid-century inspired office chair is part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. It is available in several shell colors and base finishes, with options for a plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. This chair can be customized to include or exclude armrests, making it suitable for both home and business settings. It is also qualified for contract use.</p>

<p>The chair is constructed with a 5-wheel plastic coated aluminum base, providing stability and easy movement. It features a pneumatic chair adjust for easy raise/lower action. The shell base glider is made of cast aluminum with a modified nylon PA6/PA66 coating, ensuring durability and longevity. The shell thickness is 10 mm. The seat is made of HD36 foam, providing comfort and support.</

## Load Python libraries to view HTML

In [19]:
from IPython.display import display, HTML
display(HTML(response))

Dimension,Measurement (inches)
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14


## Summarizing

In [20]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

### Summarize with a word/sentence/character limit

In [21]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


The reviewer bought a panda plush toy for their daughter's birthday. They found it soft, cute, and friendly-looking, but smaller than expected for the price. It arrived early.


### Summarize with a focus

In [22]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


The product arrived a day earlier than expected, providing a positive shipping experience for the customer.


### Try "extract" instead of "summarize"

In [23]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The panda plush toy arrived a day earlier than expected.


## Chatbot